In [1]:
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
import folium
from folium.plugins import Geocoder
from transformers import pipeline

/Users/sashkawarner/repos/oldtime-map/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-21 20:44:56.097254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Read in kml file of oldtime music gatherings
fiona.supported_drivers['KML'] = 'rw'
raw_points = gpd.read_file("./Old Time Open Jams.kml", driver="KML")
raw_points.head()

,Name,Description,geometry
0,"*Fairhope, AL",Jam n Folks meet every Tuesday (unless we are ...,POINT Z (-87.90006 30.52789 0.00000)
1,"*Anchorage, AK",Currently on hold<br>Irish session<br>Thursday...,POINT Z (-149.90654 61.19747 0.00000)
2,"*Ester, AK","The Golden Eagle Saloon<br>3630 Main St, Ester...",POINT Z (-148.01637 64.84680 0.00000)
3,"Fairbanks, AK",This jam is Zoom only until further notice. ...,POINT Z (-147.77080 64.83923 0.00000)
4,"*Fairbanks, AK",Irish Session ...,POINT Z (-147.71441 64.85332 0.00000)


In [5]:
raw_points.iloc[470,:]

Name                                               *Brooklyn, NY
Description    Dear Friends,<br>Like many people, due to the ...
geometry                      POINT Z (-74.0169398 40.6756909 0)
Name: 470, dtype: object

In [6]:
raw_points.to_csv("oldtime.csv")

In [13]:
raw_points[raw_points["Description"].str.contains("picnic", case=False)].iloc[8]["Description"]

"Fiddler's Picnic. 3 day event, first weekend in August.<br>Aug 4-6, 2023<br>https://www.facebook.com/Montana-Old-Time-Fiddlers-Picnic-403947783036154/?<br><br>Updated CR 1/2/23"

In [2]:
# Takes 2 min to download
classifier = pipeline("zero-shot-classification")
#classifier("We meet every two weeks", candidate_labels=["weekly gathering", "annual event"])

No model was supplied, defaulted to roberta-large-mnli and revision 130fb28 (https://huggingface.co/roberta-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
2023-03-21 20:46:34.350792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at roberta-large-mnli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


{'sequence': 'We meet every two weeks',
 'labels': ['weekly gathering', 'annual event'],
 'scores': [0.9827330112457275, 0.017267024144530296]}

In [14]:
test = "Fiddler's Picnic. 3 day event, first weekend in August.<br>Aug 4-6, 2023<br>https://www.facebook.com/Montana-Old-Time-Fiddlers-Picnic-403947783036154/?<br><br>Updated CR 1/2/23"
classifier(test, candidate_labels=["weekly gathering", "annual event"])

{'sequence': "Fiddler's Picnic. 3 day event, first weekend in August.<br>Aug 4-6, 2023<br>https://www.facebook.com/Montana-Old-Time-Fiddlers-Picnic-403947783036154/?<br><br>Updated CR 1/2/23",
 'labels': ['annual event', 'weekly gathering'],
 'scores': [0.5418637990951538, 0.4581362009048462]}

In [10]:
# Filter out non-oldtime gatherings
oldtime_events = raw_points.copy()
oldtime_events = oldtime_events[~oldtime_events["Description"].str.contains("irish|celtic|scottish", case=False)]
print("Number of gatherings: %d" % oldtime_events.shape[0])
def classify_event(text):
    labels = ["weekly gathering", "annual event"]
    probs = classifier(text, candidate_labels=labels)
    clf = labels[np.argmax(probs)]
    return clf

print(classify_event(oldtime_events["Description"].iloc[0]))

'''# Get first section of text before line break
oldtime_events["header"] = oldtime_events["Description"].str.replace("(?<=<br>).*", "")

# Add category column
oldtime_events["category"] = [np.nan] * oldtime_events.shape[0]

# Select specific types of events
# and categorize
jam_q = oldtime_events["header"].str.contains("jam|session", case=False)
festival_q = (oldtime_events["header"].str.contains("fest|festival(?!s)|competition|contest|annual|gathering|(oldtime|old-time) social|camp(?=(\s|<br>|out))|class(?=(\s|es))|music school", case=False) | oldtime_events["Description"].str.contains("workshop(?=(\s|s))", case=False)) & ~jam_q
festivals = oldtime_events["header"][festival_q]
oldtime_events.loc[festival_q, "category"] = "festival_or_camp"

# Assume jams represent everything left over - this seems to be true based on EDA
jam_q = ~festival_q
jams = oldtime_events["header"][jam_q]
oldtime_events.loc[jam_q, "category"] = "jam"

# Check number of elements in each category
print("Number of jams: %d" % jams.shape[0])
print("Number of festivals/camps: %d" % festivals.shape[0])'''

#print("NAs in category row: %d" % sum(oldtime_events["category"].isna()))

Number of gatherings: 1156
weekly gathering


'# Get first section of text before line break\noldtime_events["header"] = oldtime_events["Description"].str.replace("(?<=<br>).*", "")\n\n# Add category column\noldtime_events["category"] = [np.nan] * oldtime_events.shape[0]\n\n# Select specific types of events\n# and categorize\njam_q = oldtime_events["header"].str.contains("jam|session", case=False)\nfestival_q = (oldtime_events["header"].str.contains("fest|festival(?!s)|competition|contest|annual|gathering|(oldtime|old-time) social|camp(?=(\\s|<br>|out))|class(?=(\\s|es))|music school", case=False) | oldtime_events["Description"].str.contains("workshop(?=(\\s|s))", case=False)) & ~jam_q\nfestivals = oldtime_events["header"][festival_q]\noldtime_events.loc[festival_q, "category"] = "festival_or_camp"\n\n# Assume jams represent everything left over - this seems to be true based on EDA\njam_q = ~festival_q\njams = oldtime_events["header"][jam_q]\noldtime_events.loc[jam_q, "category"] = "jam"\n\n# Check number of elements in each categ

In [ ]:
for i in range(festivals.shape[0]):
    print(festivals.iloc[i])
    print('\n')

In [ ]:
for i in range(jams.shape[0]):
    print(jams.iloc[i])
    print('\n')

In [ ]:
# Create new folium map
m = folium.Map(location=[47.116386, -101.299591], zoom_start = 4)

# Create feature groups
jam_fg = folium.FeatureGroup(name="Jams")
festivals_fg = folium.FeatureGroup(name="Festivals / Camps")

# Create markers for each category and add to feature groups - color code
for i in range(oldtime_events.shape[0]):
    geom = oldtime_events["geometry"].iloc[i]
    name = oldtime_events["Name"].iloc[i]
    desc = oldtime_events["Description"].iloc[i]
    if(oldtime_events["category"].iloc[i] == "jam"):
        marker = folium.Marker(location=[geom.y, geom.x], popup=desc, icon=folium.Icon(color="green")).add_to(jam_fg)
    elif(oldtime_events["category"].iloc[i] == "festival_or_camp"):
        marker = folium.Marker(location=[geom.y, geom.x], popup=desc, icon=folium.Icon(color="red")).add_to(festivals_fg)

# Add feature group to map
jam_fg.add_to(m)
festivals_fg.add_to(m)

# Add layer control to map
m.add_child(folium.map.LayerControl())

In [ ]:
Geocoder().add_to(m)
m

In [ ]:
#m.save("Old Time Gatherings.html")